# Food Scan Benchmark


This notebook provides a complete, end-to-end workflow for benchmarking Multimodal Large Language Models (MLLMs) on January's food image dataset (JFID).

**The process is as follows:**

1.  **Setup:** Install dependencies and configure API keys.
2.  **Define Components:** Set up Pydantic schemas, the dataset loader, the model wrapper, and evaluation metrics.
3.  **Run Evaluation:** Loop through the dataset, send images to a chosen MLLM, and collect predictions.
4.  **Analyze Results:** Calculate metrics and summarize the model's performance.

The dataset is downloaded from a public S3 bucket and cached locally.


## Setup


Add your API keys to the environment:

```
echo OPENAI_API_KEY="sk-..."
echo GEMINI_API_KEY="..."
```


In [1]:
# Install packages
%pip install --upgrade litellm boto3 pandas tqdm python-dotenv pydantic tabulate openai scikit-learn scipy plotly openpyxl

/Users/amirhosseinian/January/food-scan-benchmarks/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


## Imports


In [2]:
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pydantic import BaseModel, Field
from typing import List
import openai
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import linear_sum_assignment
import hashlib
import pandas as pd
import numpy as np
import ast
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import tarfile
import litellm
from litellm.exceptions import APIError
from typing import Optional
import json
import base64
from pathlib import Path
from tqdm.auto import tqdm
import asyncio
import time
from typing import Union, Tuple
from difflib import SequenceMatcher

warnings.filterwarnings("ignore")

## Core Components


### Schema Definition


In [ ]:
class Ingredient(BaseModel):
    name: str = Field(description="Name of the ingredient, e.g., 'scrambled eggs'")
    quantity: float = Field(description="Numerical quantity of the ingredient")
    unit: str = Field(description="Unit of measurement, e.g., 'cup', 'slice', 'g'")
    calories: float = Field(description="Estimated calories for this ingredient")
    carbs: float = Field(
        description="Estimated grams of carbohydrates for this ingredient"
    )
    protein: float = Field(description="Estimated grams of protein for this ingredient")
    fat: float = Field(description="Estimated grams of fat for this ingredient")


class TotalMacros(BaseModel):
    calories: float = Field(description="Total estimated calories for the entire meal")
    carbs: float = Field(
        description="Total estimated grams of carbohydrates for the entire meal"
    )
    protein: float = Field(
        description="Total estimated grams of protein for the entire meal"
    )
    fat: float = Field(description="Total estimated grams of fat for the entire meal")


class FoodAnalysis(BaseModel):
    meal_name: str = Field(
        description="A descriptive name for the meal, e.g., 'Breakfast Platter'"
    )
    ingredients: List[Ingredient] = Field(
        description="A list of all identified ingredients and their nutritional information"
    )
    total_macros: TotalMacros = Field(
        description="The sum of macros for all ingredients"
    )


### Dataset Class


In [6]:
class FoodScanDataset:
    """Handles downloading, caching, and loading the food dataset."""

    _S3_BUCKET = "january-food-image-dataset-public"
    _S3_KEY = "food-scan-benchmark-dataset.tar.gz"

    def __init__(self, root: Path):
        self.root = root.expanduser()
        self.img_dir = self.root / "food-scan-benchmark-dataset" / "fsb_images"
        self.csv_path = (
            self.root / "food-scan-benchmark-dataset" / "food_scan_bench_v1.csv"
        )

        if not self.csv_path.exists():
            self._download_and_extract()

        self.df = pd.read_csv(self.csv_path)

    def _download_and_extract(self):
        print(f"Dataset not found in {self.root}. Downloading from S3...")
        self.root.mkdir(parents=True, exist_ok=True)
        local_archive = self.root / "fsb.tar.gz"

        s3 = boto3.client(
            "s3",
            config=Config(signature_version=UNSIGNED),
        )
        with open(local_archive, "wb") as f:
            s3.download_fileobj(self._S3_BUCKET, self._S3_KEY, f)

        print("Download complete. Extracting...")
        with tarfile.open(local_archive) as tar:
            tar.extractall(path=self.root)
        local_archive.unlink()
        print("Extraction complete.")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        img_path = self.img_dir / row.image_filename

        try:
            ingredients = ast.literal_eval(row.ingredients_list)
        except (ValueError, SyntaxError):
            ingredients = []

        return {
            "image_id": row.image_id,
            "image_path": img_path,
            "gt": {
                "meal_name": row.meal_name,
                "ingredients": ingredients,
                "macros": {
                    "calories": row.total_calories,
                    "carbs": row.total_carbs,
                    "protein": row.total_protein,
                    "fat": row.total_fat,
                },
            },
        }

### Model config and helpers


In [ ]:
MODEL_COSTS = {
    "gpt-4.1": {
        "input": 2.00,
        "output": 8.00,
        "display_name": "gpt-4.1",
    },
    "gpt-4o": {
        "input": 2.50,
        "output": 10.00,
        "display_name": "gpt-4o",
    },
    "gpt-4o-mini": {
        "input": 0.15,
        "output": 0.60,
        "display_name": "gpt-4o-mini",
    },
    "gemini/gemini-2.5-flash-preview-05-20": {
        "input": 0.15,
        "output": 0.60,
        "display_name": "gemini-2.5-flash",
    },
    "gemini/gemini-2.5-pro-preview-06-05": {
        "input": 1.25,
        "output": 10.00,
        "display_name": "gemini-2.5-pro",
    },
}


def img2b64(path: Path) -> str:
    """Converts an image file to a base64 encoded string for API calls."""
    encoded = base64.b64encode(path.read_bytes()).decode()
    return f"data:image/jpeg;base64,{encoded}"


def calculate_cost(model_name: str, input_tokens: int, output_tokens: int) -> float:
    """Calculate the cost for a model based on token usage."""
    if model_name not in MODEL_COSTS:
        return 0.0

    costs = MODEL_COSTS[model_name]
    input_cost = (input_tokens / 1_000_000) * costs["input"]
    output_cost = (output_tokens / 1_000_000) * costs["output"]
    return round(input_cost + output_cost, 6)


def get_display_name(model_name: str) -> str:
    """Return the user-friendly model label (falls back to raw id)."""
    return MODEL_COSTS.get(model_name, {}).get("display_name", model_name)

### Model Wrapper


In [146]:
class LiteModel:
    """A robust wrapper around any LiteLLM-supported vision model with prompt engineering options."""

    PROMPT_VARIANTS = {
        "detailed": {
            "suffix": "d",
            "prompt": "You are an expert nutritionist with 20 years of experience. Analyze this food image very carefully and provide the most accurate breakdown possible. Consider portion sizes, cooking methods, and hidden ingredients.",
        },
        "step_by_step": {
            "suffix": "s",
            "prompt": "You are an expert nutritionist. Please analyze this image step by step: 1) First identify all visible food items, 2) Estimate portion sizes, 3) Calculate nutritional content for each item, 4) Sum the totals. Be precise and systematic.",
        },
        "conservative": {
            "suffix": "c",
            "prompt": "You are a conservative nutritionist who prefers to underestimate rather than overestimate. Analyze this food image and provide a realistic, slightly conservative nutritional breakdown.",
        },
        "confident": {
            "suffix": "f",
            "prompt": "You are a highly confident nutritionist. Analyze this food image and provide your best estimate of the nutritional content. Trust your expertise.",
        },
    }

    def __init__(self, model_name: str, prompt_variant="detailed", **litellm_kwargs):
        self.model_name = model_name
        self.prompt_variant = prompt_variant
        self.kwargs = litellm_kwargs

        cfg = self.PROMPT_VARIANTS.get(prompt_variant, self.PROMPT_VARIANTS["detailed"])
        self.system_prompt = cfg["prompt"]
        self.prompt_suffix = cfg["suffix"]

    async def analyse(self, img_path: Path) -> Tuple[Optional[dict], Optional[str]]:
        """
        Analyzes an image and returns a structured dict with cost info, or None and an error message on failure.

        Returns:
            Tuple[Optional[dict], Optional[str]]: A tuple of (result, error_message).
                                                  On success, result is a dict and error_message is None.
                                                  On failure, result is None and error_message is a string.
        """
        b64_img = img2b64(img_path)

        messages = [
            {"role": "system", "content": self.system_prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Analyze this food image and provide a detailed nutritional breakdown. Include: meal name, all ingredients with quantities/units, and complete macro information (calories, carbs, protein, fat) for each ingredient and the total meal.",
                    },
                    {"type": "image_url", "image_url": {"url": b64_img}},
                ],
            },
        ]

        try:
            resp = await litellm.acompletion(
                model=self.model_name,
                messages=messages,
                response_format=FoodAnalysis,
                temperature=0.0,
                **self.kwargs,
            )
            raw = resp.choices[0].message.content.strip()
            data = json.loads(raw)

            usage = resp.usage
            input_tokens = usage.prompt_tokens if usage else 0
            output_tokens = usage.completion_tokens if usage else 0
            cost = calculate_cost(self.model_name, input_tokens, output_tokens)

            result = FoodAnalysis(**data).model_dump()
            result["cost_usd"] = cost
            result["prompt_variant"] = self.prompt_variant

            return result, None

        except APIError as e:
            error_msg = f"API Error: {e}"
            print(f"{error_msg} for {img_path.name}")
            return None, error_msg
        except Exception as e:
            error_msg = f"Unexpected Error: {e}"
            print(f"{error_msg} for {img_path.name}")
            return None, error_msg


def pretty_label(full_model_name: str) -> str:
    for variant, meta in LiteModel.PROMPT_VARIANTS.items():
        postfix = f"_{variant}"
        if full_model_name.endswith(postfix):
            base = full_model_name[: -len(postfix)]
            return f"{get_display_name(base)}_{meta['suffix']}"

    if "_" in full_model_name:
        base, variant = full_model_name.rsplit("_", 1)
        suffix = LiteModel.PROMPT_VARIANTS.get(variant, {}).get(
            "suffix", variant[0].lower()
        )
        return f"{get_display_name(base)}_{suffix}"

    return get_display_name(full_model_name)

### Metrics


In [147]:
class Metrics:
    """Comprehensive metrics for food analysis evaluation."""

    _embedding_cache = {}

    @staticmethod
    def _normalize_ingredient_list(ingredients: Union[str, list, None]) -> List[dict]:
        """
        IMPROVEMENT: Centralized cleanup function.
        Safely parses ingredient data which might be a string representation of a list.
        This avoids repeating the same try-except block in multiple metric functions.
        """
        if not ingredients:
            return []
        if isinstance(ingredients, list):
            return ingredients
        if isinstance(ingredients, str):
            try:
                parsed = ast.literal_eval(ingredients)
                return parsed if isinstance(parsed, list) else []
            except (ValueError, SyntaxError):
                return []
        return []

    @staticmethod
    async def get_embedding(text, model="text-embedding-3-small"):
        """Get OpenAI embedding with caching."""
        cache_key = hashlib.md5(f"{text}_{model}".encode()).hexdigest()
        if cache_key in Metrics._embedding_cache:
            return Metrics._embedding_cache[cache_key]
        try:
            client = openai.AsyncOpenAI()
            response = await client.embeddings.create(model=model, input=text.strip())
            embedding = response.data[0].embedding
            Metrics._embedding_cache[cache_key] = embedding
            return embedding
        except Exception as e:
            print(f"Error getting embedding for '{text}': {e}")
            return [0.0] * 1536

    @staticmethod
    async def semantic_ingredient_match_embeddings(
        gt_ingredients, pred_ingredients, threshold=0.75
    ):
        """Semantic ingredient matching using OpenAI embeddings and cosine similarity."""
        gt_ingredients = Metrics._normalize_ingredient_list(gt_ingredients)
        pred_ingredients = Metrics._normalize_ingredient_list(pred_ingredients)

        def normalize_name(item):
            return str(item.get("name", "")).lower().strip()

        gt_names = [normalize_name(x) for x in gt_ingredients if normalize_name(x)]
        pred_names = [normalize_name(x) for x in pred_ingredients if normalize_name(x)]

        if not gt_names and not pred_names:
            return 1.0, []
        if not gt_names or not pred_names:
            return 0.0, []

        gt_embeddings = await asyncio.gather(
            *(Metrics.get_embedding(name) for name in gt_names)
        )
        pred_embeddings = await asyncio.gather(
            *(Metrics.get_embedding(name) for name in pred_names)
        )

        gt_embeddings = [emb for emb in gt_embeddings if emb and len(emb) > 1]
        pred_embeddings = [emb for emb in pred_embeddings if emb and len(emb) > 1]

        if not gt_embeddings or not pred_embeddings:
            return 0.0, []

        similarity_matrix = cosine_similarity(gt_embeddings, pred_embeddings)
        cost_matrix = 1 - similarity_matrix
        row_indices, col_indices = linear_sum_assignment(cost_matrix)

        matches = 0
        match_details = []
        for row_idx, col_idx in zip(row_indices, col_indices):
            similarity = similarity_matrix[row_idx, col_idx]
            if similarity >= threshold:
                matches += 1
                match_details.append(
                    {
                        "gt_ingredient": gt_names[row_idx],
                        "pred_ingredient": pred_names[col_idx],
                        "similarity": similarity,
                    }
                )

        recall = matches / len(gt_names)
        return recall, match_details

    @staticmethod
    def semantic_ingredient_match(gt_ingredients, pred_ingredients, threshold=0.7):
        """Fallback method using string similarity."""
        gt_ingredients = Metrics._normalize_ingredient_list(gt_ingredients)
        pred_ingredients = Metrics._normalize_ingredient_list(pred_ingredients)

        def normalize_name(item):
            name_str = item.get("name", "") if isinstance(item, dict) else item
            return str(name_str).lower().strip().replace("-", " ").replace("_", " ")

        gt_names = [normalize_name(x) for x in gt_ingredients if normalize_name(x)]
        pred_names = [normalize_name(x) for x in pred_ingredients if normalize_name(x)]

        if not gt_names:
            return 1.0 if not pred_names else 0.0

        matches = 0
        for gt_name in gt_names:
            if pred_names:
                best_match = max(
                    SequenceMatcher(None, gt_name, pred_name).ratio()
                    for pred_name in pred_names
                )
                if best_match >= threshold:
                    matches += 1

        return matches / len(gt_names)

    @staticmethod
    def ingredients_f1(gt, pred):
        """F-score on ingredient names (case-insensitive)."""
        gt_list = Metrics._normalize_ingredient_list(gt)
        pred_list = Metrics._normalize_ingredient_list(pred)

        def _name(x):
            return (
                str(x.get("name", "")).lower().strip()
                if isinstance(x, dict)
                else str(x).lower().strip()
            )

        g_names = {_name(x) for x in gt_list if _name(x)}
        p_names = {_name(x) for x in pred_list if _name(x)}

        if not g_names and not p_names:
            return 1.0
        if not g_names or not p_names:
            return 0.0

        tp = len(g_names & p_names)
        if tp == 0:
            return 0.0

        precision = tp / len(p_names)
        recall = tp / len(g_names)
        return 2 * precision * recall / (precision + recall)

    @staticmethod
    async def semantic_f1_score(gt_ingredients, pred_ingredients, threshold=0.75):
        """
        NEW: Calculates an F1 score based on semantic matches from embeddings.
        This measures a balance of semantic recall and precision.
        """
        gt_list = Metrics._normalize_ingredient_list(gt_ingredients)
        pred_list = Metrics._normalize_ingredient_list(pred_ingredients)

        def normalize_name(item):
            return str(item.get("name", "")).lower().strip()

        gt_names = [normalize_name(x) for x in gt_list if normalize_name(x)]
        pred_names = [normalize_name(x) for x in pred_list if normalize_name(x)]

        if not gt_names and not pred_names:
            return 1.0
        if not gt_names or not pred_names:
            return 0.0

        _, match_details = await Metrics.semantic_ingredient_match_embeddings(
            gt_list, pred_list, threshold
        )

        matches = len(match_details)

        precision = matches / len(pred_names)
        recall = matches / len(gt_names)

        if precision + recall == 0:
            return 0.0

        f1_score = 2 * (precision * recall) / (precision + recall)
        return f1_score

    @staticmethod
    def macro_mae(gt_mac: dict, pred_mac: dict):
        """Calculates Mean Absolute Error over the four main macros."""
        keys = ["calories", "carbs", "protein", "fat"]
        errors = [abs(gt_mac.get(k, 0) - pred_mac.get(k, 0)) for k in keys]
        return float(np.mean(errors))

    @staticmethod
    def macro_percentage_error(gt_mac, pred_mac):
        """Calculate percentage error for each macro."""
        keys = ["calories", "carbs", "protein", "fat"]
        errors = {}
        for key in keys:
            gt_val = gt_mac.get(key, 0)
            pred_val = pred_mac.get(key, 0)
            if gt_val > 0:
                errors[key] = abs(gt_val - pred_val) / gt_val * 100
            else:
                errors[key] = 0 if pred_val == 0 else 100
        return errors

    @staticmethod
    def ingredient_count_accuracy(gt_ingredients, pred_ingredients):
        """How well does the model predict the number of ingredients?"""
        gt_count = len(Metrics._normalize_ingredient_list(gt_ingredients))
        pred_count = len(Metrics._normalize_ingredient_list(pred_ingredients))
        if gt_count == 0 and pred_count == 0:
            return 1.0
        if gt_count == 0:
            return 0.0
        return 1 - abs(gt_count - pred_count) / gt_count

## Evaluation Function


In [ ]:
async def _process_sample(
    idx: int,
    ds: FoodScanDataset,
    llm: LiteModel,
    model_name: str,
    use_embeddings: bool = True,
) -> dict:
    start_time = time.time()
    sample = ds[idx]
    pred, error_msg = await llm.analyse(sample["image_path"])
    end_time = time.time()
    gt = sample["gt"]

    item = {
        "image_id": sample["image_id"],
        "model": model_name,
        "response_time_seconds": round(end_time - start_time, 2),
    }

    if pred is None:
        item.update(
            {
                "f1_ing": 0.0,
                "semantic_match": 0.0,
                "semantic_f1_ing": 0.0,
                "semantic_match_embeddings": 0.0,
                "ingredient_count_acc": 0.0,
                "mae_mac": None,
                "error": error_msg or "failed",
                "cost_usd": 0.0,
                "calories_pct_error": None,
                "carbs_pct_error": None,
                "protein_pct_error": None,
                "fat_pct_error": None,
                "match_details": None,
            }
        )
    else:
        gt_ingredients = gt["ingredients"]
        pred_ingredients = pred.get("ingredients", [])
        gt_macros = gt["macros"]
        pred_macros = pred.get("total_macros", {})

        semantic_match_embeddings, match_details = 0.0, None
        semantic_f1 = 0.0
        if use_embeddings:
            try:
                (
                    semantic_match_embeddings,
                    match_details,
                ) = await Metrics.semantic_ingredient_match_embeddings(
                    gt_ingredients, pred_ingredients
                )
                semantic_f1 = await Metrics.semantic_f1_score(
                    gt_ingredients, pred_ingredients
                )
            except Exception as e:
                print(
                    f"Error in embedding similarity for image {sample['image_id']}: {e}"
                )
                semantic_match_embeddings = Metrics.semantic_ingredient_match(
                    gt_ingredients, pred_ingredients
                )
                semantic_f1 = 0.0

        pct_errors = Metrics.macro_percentage_error(gt_macros, pred_macros)

        item.update(
            {
                "f1_ing": Metrics.ingredients_f1(gt_ingredients, pred_ingredients),
                "semantic_match": Metrics.semantic_ingredient_match(
                    gt_ingredients, pred_ingredients
                ),
                "semantic_f1_ing": semantic_f1,
                "semantic_match_embeddings": semantic_match_embeddings,
                "ingredient_count_acc": Metrics.ingredient_count_accuracy(
                    gt_ingredients, pred_ingredients
                ),
                "mae_mac": Metrics.macro_mae(gt_macros, pred_macros),
                "error": None,
                "cost_usd": pred.get("cost_usd", 0.0),
                "calories_pct_error": pct_errors.get("calories"),
                "carbs_pct_error": pct_errors.get("carbs"),
                "protein_pct_error": pct_errors.get("protein"),
                "fat_pct_error": pct_errors.get("fat"),
                "match_details": match_details,
            }
        )

    return item


async def run_evaluation(
    models: Union[str, List[str]],
    cache_dir: Path,
    max_items: Optional[int] = None,
    max_concurrent: int = 5,
    prompt_variants: Optional[List[str]] = None,
    use_embeddings: bool = True,
) -> pd.DataFrame:
    """Run evaluation with multiple models and prompt variants."""
    models = [models] if isinstance(models, str) else models
    prompt_variants = prompt_variants or ["detailed"]

    ds = FoodScanDataset(cache_dir)
    n = min(max_items, len(ds)) if max_items else len(ds)
    all_results = []

    for prompt_variant in prompt_variants:
        print(f"\n=== Running with prompt variant: '{prompt_variant}' ===")
        print(
            f"Using {'OpenAI embeddings' if use_embeddings else 'string-based'} semantic similarity"
        )

        async def _run_model_with_prompt(model_name: str, position: int):
            llm = LiteModel(model_name, prompt_variant=prompt_variant)
            sem = asyncio.Semaphore(max_concurrent)
            pbar = tqdm(
                total=n,
                desc=f"{model_name} ({prompt_variant})".ljust(40),
                position=position,
                leave=True,
                dynamic_ncols=True,
                colour="green",
                bar_format="{desc}│{bar}│ {n_fmt}/{total_fmt}",
            )

            async def _worker(i: int):
                async with sem:
                    result = await _process_sample(
                        i, ds, llm, f"{model_name}_{prompt_variant}", use_embeddings
                    )
                    pbar.update(1)
                    return result

            results = await asyncio.gather(*(_worker(i) for i in range(n)))
            pbar.close()
            return results

        model_tasks = [
            _run_model_with_prompt(model, i) for i, model in enumerate(models)
        ]
        nested_results = await asyncio.gather(*model_tasks)
        all_results.extend(item for sublist in nested_results for item in sublist)

    return pd.DataFrame(all_results)

## Analysis and Visualization


In [ ]:
class BenchmarkAnalyzer:
    """Comprehensive analysis and visualization of benchmark results."""

    def __init__(self, results_df):
        self.df = results_df.copy()
        self.df["pretty_model"] = self.df["model"].apply(pretty_label)
        if not self.df.empty:
            self.successful_df = self.df[self.df["error"].isna()].copy()
        else:
            self.successful_df = pd.DataFrame()

    def summary_statistics(self):
        """Generate comprehensive summary statistics."""
        print("=== BENCHMARK SUMMARY ===\n")
        if self.df.empty:
            print("No results to analyze.")
            return

        for model in sorted(self.df["model"].unique()):
            display_name = pretty_label(model)
            model_df = self.df[self.df["model"] == model]
            successful = model_df[model_df["error"].isna()]
            success_rate = (len(successful) / len(model_df)) * 100

            print(f"--- {display_name} ---")
            print(
                f"  Success Rate: {success_rate:.1f}% ({len(successful)}/{len(model_df)})"
            )

            if not successful.empty:
                print(
                    f"  Avg Semantic Match (Embeddings): {successful['semantic_match_embeddings'].mean():.3f}"
                )
                print(
                    f"  Avg F1 Score (Ingredients): {successful['f1_ing'].mean():.3f}"
                )
                print(f"  Avg MAE (Macros): {successful['mae_mac'].mean():.1f}")
                print(f"  Avg Cost per Image: ${successful['cost_usd'].mean():.4f}")
                print(
                    f"  Avg Response Time: {successful['response_time_seconds'].mean():.1f}s\n"
                )

        self.analyze_errors()

    def analyze_errors(self):
        """Analyze and summarize the specific errors encountered."""
        print("--- ERROR ANALYSIS ---")
        error_df = self.df[self.df["error"].notna()]
        if error_df.empty:
            print("No errors encountered. All API calls were successful.\n")
            return

        print("Error counts by model:")
        error_summary = error_df.groupby("model")["error"].count().sort_index()
        print(error_summary.to_string())

        print("\nMost common error messages:")
        common_errors = (
            error_df["error"].str.split(":").str[0].value_counts().nlargest(5)
        )
        print(common_errors.to_string())
        print()

    def analyze_ingredient_matches(self):
        """Analyze detailed ingredient matching results."""
        if (
            self.successful_df.empty
            or "match_details" not in self.successful_df.columns
        ):
            print("No match details available.")
            return

        print("=== INGREDIENT MATCHING ANALYSIS ===\n")
        all_matches = self.successful_df.explode("match_details").dropna(
            subset=["match_details"]
        )
        if all_matches.empty:
            print("No successful ingredient matches found.")
            return

        matches_df = pd.json_normalize(all_matches["match_details"])

        matches_df["model"] = all_matches["model"].values

        print(f"Total successful ingredient matches: {len(matches_df)}")
        print(
            f"Average similarity score: {matches_df['similarity'].mean():.3f} (std: {matches_df['similarity'].std():.3f})"
        )

        print("\n--- TOP 10 INGREDIENT MATCHES ---")
        print(
            matches_df.nlargest(10, "similarity")[
                ["gt_ingredient", "pred_ingredient", "similarity", "model"]
            ].to_string(index=False)
        )

        print("\n--- 10 MOST CHALLENGING MATCHES (Lowest Similarity) ---")
        challenging = matches_df[matches_df["similarity"] < 0.85].nsmallest(
            10, "similarity"
        )
        if not challenging.empty:
            print(
                challenging[
                    ["gt_ingredient", "pred_ingredient", "similarity", "model"]
                ].to_string(index=False)
            )
        else:
            print("No challenging matches found (all similarities >= 0.85).")

    def statistical_significance_test(self, metric="semantic_match_embeddings"):
        """Test statistical significance between models using t-test."""
        if self.successful_df.empty or metric not in self.successful_df.columns:
            print(f"Metric '{metric}' not available for significance testing.")
            return

        models = sorted(self.successful_df["model"].unique())
        if len(models) < 2:
            return

        print(f"\n=== STATISTICAL SIGNIFICANCE (T-TEST on {metric}) ===\n")
        for i, model1 in enumerate(models):
            for model2 in models[i + 1 :]:
                data1 = self.successful_df[self.successful_df["model"] == model1][
                    metric
                ]
                data2 = self.successful_df[self.successful_df["model"] == model2][
                    metric
                ]

                if len(data1) > 1 and len(data2) > 1:
                    stat, p_value = stats.ttest_ind(data1, data2, equal_var=False)
                    significance = (
                        "***"
                        if p_value < 0.001
                        else "**"
                        if p_value < 0.01
                        else "*"
                        if p_value < 0.05
                        else "ns"
                    )
                    print(
                        f"{pretty_label(model1).ljust(30)} vs. "
                        f"{pretty_label(model2).ljust(30)} | "
                        f"p-value: {p_value:.4f} {significance}"
                    )
        print()

    def create_performance_dashboard(self):
        """Create a comprehensive performance comparison dashboard with improved styling."""
        if self.successful_df.empty:
            print("No successful predictions to plot.")
            return

        models = sorted(self.df["model"].unique())

        short_labels = {m: pretty_label(m) for m in models}
        idx_map = {m: i + 1 for i, m in enumerate(short_labels)}

        colors = [
            "#1f77b4",
            "#ff7f0e",
            "#2ca02c",
            "#d62728",
            "#9467bd",
            "#8c564b",
            "#e377c2",
            "#7f7f7f",
            "#bcbd22",
            "#17becf",
        ]

        metric_col = "semantic_match_embeddings"

        fig = make_subplots(
            rows=2,
            cols=3,
            subplot_titles=(
                "Embedding Similarity Distribution",
                "Lexical F1 Score (Ingredients)",
                "Embedding F1 Score (Ingredients)",
                "Macro Nutritional MAE",
                "Cost vs. Performance",
                "Response Time Distribution",
            ),
            vertical_spacing=0.15,
            horizontal_spacing=0.08,
        )

        for i, model in enumerate(short_labels):
            m_idx = str(idx_map[model])
            color = colors[i % len(colors)]
            d_ok = self.successful_df[self.successful_df["model"] == model]

            box_style = dict(
                marker_color=color,
                marker_line_color="rgba(0,0,0,0.3)",
                marker_line_width=1,
                fillcolor=f"rgba({','.join(str(int(color[i : i + 2], 16)) for i in (1, 3, 5))}, 0.6)",
                line_color=color,
                line_width=2,
            )

            fig.add_trace(
                go.Box(
                    y=d_ok["semantic_match_embeddings"],
                    name=m_idx,
                    legendgroup=m_idx,
                    showlegend=False,
                    **box_style,
                ),
                row=1,
                col=1,
            )

            fig.add_trace(
                go.Box(
                    y=d_ok["f1_ing"],
                    name=m_idx,
                    legendgroup=m_idx,
                    showlegend=False,
                    **box_style,
                ),
                row=1,
                col=2,
            )

            fig.add_trace(
                go.Box(
                    y=d_ok["semantic_f1_ing"],
                    name=m_idx,
                    legendgroup=m_idx,
                    showlegend=False,
                    **box_style,
                ),
                row=1,
                col=3,
            )

            fig.add_trace(
                go.Box(
                    y=d_ok["mae_mac"],
                    name=m_idx,
                    legendgroup=m_idx,
                    showlegend=False,
                    **box_style,
                ),
                row=2,
                col=1,
            )

            summary_df = (
                self.successful_df.groupby("model")
                .agg(
                    avg_cost=("cost_usd", "mean"),
                    avg_performance=(metric_col, "mean"),
                    avg_time=("response_time_seconds", "mean"),
                    performance_std=(metric_col, "std"),
                )
                .reindex(models)
                .reset_index()
            )

            summary_df["short_label"] = summary_df["model"].map(short_labels)
            summary_df["m_idx"] = summary_df["model"].map(idx_map).astype(str)

            min_time = summary_df["avg_time"].min()
            max_time = summary_df["avg_time"].max()

            if max_time == min_time:
                summary_df["normalized_speed"] = 1.0
            else:
                summary_df["normalized_speed"] = (max_time - summary_df["avg_time"]) / (
                    max_time - min_time
                )

            base_size = 15
            scaling_factor = 35
            summary_df["circle_size"] = base_size + (
                summary_df["normalized_speed"] * scaling_factor
            )

            fig.add_trace(
                go.Scatter(
                    x=summary_df["avg_cost"],
                    y=summary_df["avg_performance"],
                    mode="markers+text",
                    marker=dict(
                        size=summary_df["circle_size"],
                        color=[colors[i % len(colors)] for i in range(len(summary_df))],
                        sizemode="diameter",
                        sizemin=10,
                        line=dict(width=2, color="rgba(0,0,0,0.4)"),
                        opacity=0.8,
                    ),
                    text=summary_df["m_idx"],
                    textfont=dict(color="white", size=10, family="Arial Black"),
                    textposition="middle center",
                    hovertemplate="<b>%{customdata[0]}</b><br>"
                    + "Avg. Performance: %{y:.3f}<br>"
                    + "Avg. Cost: $%{x:.4f}<br>"
                    + "Avg. Response Time: %{customdata[1]:.2f}s<br>"
                    + "<extra></extra>",
                    customdata=summary_df[["short_label", "avg_time"]],
                    showlegend=False,
                ),
                row=2,
                col=2,
            )

            fig.update_traces(
                error_y=dict(
                    type="data",
                    array=summary_df["performance_std"],
                    visible=True,
                    thickness=1.5,
                    width=3,
                    color="rgba(0,0,0,0.3)",
                ),
                selector=dict(type="scatter"),
                row=2,
                col=2,
            )

            fig.add_trace(
                go.Box(
                    y=d_ok["response_time_seconds"],
                    name=m_idx,
                    legendgroup=m_idx,
                    showlegend=False,
                    **box_style,
                ),
                row=2,
                col=3,
            )

        for i, model in enumerate(short_labels):
            m_idx = str(idx_map[model])
            fig.add_trace(
                go.Scatter(
                    x=[None],
                    y=[None],
                    mode="markers",
                    marker=dict(
                        size=12,
                        color=colors[i % len(colors)],
                        symbol="circle",
                        line=dict(width=2, color="rgba(0,0,0,0.3)"),
                    ),
                    legendgroup=m_idx,
                    showlegend=True,
                    name=f"{m_idx}: {short_labels[model]}",
                )
            )

        axis_style = dict(
            showgrid=True,
            gridcolor="rgba(128,128,128,0.2)",
            gridwidth=1,
            zeroline=True,
            zerolinecolor="rgba(128,128,128,0.4)",
            zerolinewidth=1,
            tickfont=dict(size=11, color="#2f2f2f"),
        )

        for row in [1, 2]:
            for col in [1, 2, 3]:
                fig.update_xaxes(**axis_style, row=row, col=col)
                fig.update_yaxes(**axis_style, row=row, col=col)

        fig.update_yaxes(title_text="Similarity Score (Recall)", row=1, col=1)
        fig.update_yaxes(title_text="Lexical F1 Score", row=1, col=2)
        fig.update_yaxes(title_text="Embedding F1 Score", row=1, col=3)
        fig.update_yaxes(title_text="Mean Absolute Error", row=2, col=1)
        fig.update_xaxes(title_text="Cost per Image ($)", row=2, col=2)
        fig.update_yaxes(title_text="Similarity Score", row=2, col=2)
        fig.update_yaxes(title_text="Response Time (sec)", row=2, col=3)

        fig.update_layout(
            height=900,
            width=1400,
            title=dict(
                text="<b>Model Performance Dashboard</b>",
                x=0.5,
                xanchor="center",
                font=dict(size=24, color="#1f1f1f", family="Arial Black"),
            ),
            showlegend=True,
            legend=dict(
                title="<b>Models</b>",
                title_font=dict(size=14, color="#1f1f1f"),
                font=dict(size=11),
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="rgba(128,128,128,0.5)",
                borderwidth=1,
                x=1.02,
                y=1,
                xanchor="left",
                yanchor="top",
            ),
            plot_bgcolor="rgba(248,249,250,0.8)",
            paper_bgcolor="white",
            font=dict(family="Arial, sans-serif", size=11, color="#2f2f2f"),
            margin=dict(l=80, r=150, t=120, b=80),
            hovermode="closest",
        )

        config = dict(
            displayModeBar=True,
            displaylogo=False,
            modeBarButtonsToRemove=["pan2d", "lasso2d"],
            toImageButtonOptions=dict(
                format="png",
                filename="model_performance_dashboard",
                height=900,
                width=1400,
                scale=2,
            ),
        )

        fig.show(config=config)
        fig.write_html("performance_dashboard.html", config=config)

    def macro_accuracy_heatmap(self):
        """Create a heatmap of macro percentage error by model."""
        if self.successful_df.empty:
            return
        macro_cols = [
            "calories_pct_error",
            "carbs_pct_error",
            "protein_pct_error",
            "fat_pct_error",
        ]
        heatmap_data = (
            self.successful_df.groupby("model")[macro_cols].mean().sort_index()
        )

        heatmap_data.index = [pretty_label(m) for m in heatmap_data.index]

        fig = px.imshow(
            heatmap_data,
            labels=dict(x="Macro Nutrient", y="Model", color="Avg % Error"),
            x=["Calories", "Carbs", "Protein", "Fat"],
            y=heatmap_data.index,
            text_auto=True,
            aspect="auto",
            color_continuous_scale="RdYlGn_r",
            title="<b>Average Percentage Error by Macro (Lower is Better)</b>",
        )
        fig.update_traces(texttemplate="%{z}%")
        fig.show()

    def export_detailed_report(self, filename="benchmark_report.csv"):
        """Export a detailed report with raw data and summary to CSV files."""
        print(f"Exporting detailed report to {filename}...")
        self.df.to_csv(filename, index=False)

        if not self.successful_df.empty:
            summary = (
                self.df.groupby("model")
                .agg(
                    success_rate=("error", lambda x: x.isna().mean()),
                    semantic_match_embeddings_mean=(
                        "semantic_match_embeddings",
                        "mean",
                    ),
                    f1_ing_mean=("f1_ing", "mean"),
                    mae_mac_mean=("mae_mac", "mean"),
                    cost_usd_total=("cost_usd", "sum"),
                    response_time_seconds_mean=("response_time_seconds", "mean"),
                )
                .reset_index()
            )
            summary_filename = filename.replace(".csv", "_summary.csv")
            summary.to_csv(summary_filename, index=False)
        print("Export complete.")

    def create_win_loss_analysis(self, baseline_model_name: Optional[str] = None):
        """
        Win-Tie-Loss analysis comparing average performance across all prompt variants.

        This method:
        1. Averages the macro errors across all prompt variants for each model/image
        2. Compares these averaged performances between models
        3. Shows win/tie/loss statistics where lower error = win
        """
        print("\n=== WIN-TIE-LOSS ANALYSIS (Average Across Prompt Variants) ===")
        if self.successful_df.empty:
            print("No successful results to analyze.")
            return

        df = self.successful_df.copy()

        df[["base_model", "prompt_variant"]] = df["model"].str.split(
            "_", n=1, expand=True
        )
        base_models = sorted(df["base_model"].unique())

        if len(base_models) < 2:
            print("Need at least two models to perform a win-loss comparison.")
            return

        if baseline_model_name:
            if baseline_model_name not in base_models:
                print(
                    f"Error: Baseline model '{baseline_model_name}' not found in results."
                )
                print(f"Available models are: {base_models}")
                return
            baseline_model = baseline_model_name
        else:
            baseline_model = base_models[0]
            print(
                f"INFO: No baseline model specified. Using '{baseline_model}' as the default for win-loss analysis."
            )

        competitor_models = [m for m in base_models if m != baseline_model]

        macro_cols = [
            "calories_pct_error",
            "carbs_pct_error",
            "protein_pct_error",
            "fat_pct_error",
        ]

        avg_errors = (
            df.groupby(["image_id", "base_model"])[macro_cols].mean().reset_index()
        )

        win_loss_data = {}
        for competitor in competitor_models:
            win_loss_data[competitor] = {
                m: {"win": 0, "tie": 0, "loss": 0} for m in macro_cols
            }

            baseline_data = avg_errors[avg_errors["base_model"] == baseline_model]
            competitor_data = avg_errors[avg_errors["base_model"] == competitor]

            comparison_df = pd.merge(
                baseline_data,
                competitor_data,
                on="image_id",
                suffixes=("_base", "_comp"),
            )

            if comparison_df.empty:
                print(f"No common images between {baseline_model} and {competitor}")
                continue

            for macro in macro_cols:
                base_error = comparison_df[f"{macro}_base"]
                comp_error = comparison_df[f"{macro}_comp"]

                win_loss_data[competitor][macro]["win"] = (
                    base_error < comp_error
                ).sum()
                win_loss_data[competitor][macro]["tie"] = (
                    base_error == comp_error
                ).sum()
                win_loss_data[competitor][macro]["loss"] = (
                    base_error > comp_error
                ).sum()

        if not win_loss_data:
            print("No common images found to compare models.")
            return

        colors = {
            "win": "#16A085",
            "tie": "#95A5A6",
            "loss": "#E74C3C",
        }

        macro_titles = ["Calories", "Carbohydrates", "Protein", "Fat"]

        fig = make_subplots(
            rows=len(competitor_models),
            cols=4,
            subplot_titles=macro_titles,
            shared_xaxes=True,
            vertical_spacing=0.08,
            horizontal_spacing=0.08,
        )

        for i, competitor in enumerate(competitor_models):
            pretty_comp = pretty_label(competitor)
            row_idx = i + 1

            for j, macro in enumerate(macro_cols):
                col_idx = j + 1
                data = win_loss_data[competitor][macro]
                total = sum(data.values())

                if total == 0:
                    continue

                win_pct = data["win"] / total * 100
                tie_pct = data["tie"] / total * 100
                loss_pct = data["loss"] / total * 100

                fig.add_trace(
                    go.Bar(
                        name="Win",
                        x=[win_pct],
                        y=[pretty_comp],
                        orientation="h",
                        marker=dict(
                            color=colors["win"],
                            line=dict(color="rgba(255,255,255,0.3)", width=1),
                        ),
                        showlegend=False,
                        text=f"{win_pct:.0f}%" if win_pct > 5 else "",
                        textposition="inside",
                        textfont=dict(
                            color="white", size=10, family="Inter, Arial, sans-serif"
                        ),
                        customdata=[data["win"]],
                        hovertemplate="<b>Win</b><br>"
                        + "Count: %{customdata}<br>"
                        + "Percentage: %{x:.1f}%<br>"
                        + "<extra></extra>",
                    ),
                    row=row_idx,
                    col=col_idx,
                )

                fig.add_trace(
                    go.Bar(
                        name="Tie",
                        x=[tie_pct],
                        y=[pretty_comp],
                        orientation="h",
                        marker=dict(
                            color=colors["tie"],
                            line=dict(color="rgba(255,255,255,0.3)", width=1),
                        ),
                        showlegend=False,
                        text=f"{tie_pct:.0f}%" if tie_pct > 5 else "",
                        textposition="inside",
                        textfont=dict(
                            color="white", size=10, family="Inter, Arial, sans-serif"
                        ),
                        customdata=[data["tie"]],
                        hovertemplate="<b>Tie</b><br>"
                        + "Count: %{customdata}<br>"
                        + "Percentage: %{x:.1f}%<br>"
                        + "<extra></extra>",
                    ),
                    row=row_idx,
                    col=col_idx,
                )

                fig.add_trace(
                    go.Bar(
                        name="Loss",
                        x=[loss_pct],
                        y=[pretty_comp],
                        orientation="h",
                        marker=dict(
                            color=colors["loss"],
                            line=dict(color="rgba(255,255,255,0.3)", width=1),
                        ),
                        showlegend=False,
                        text=f"{loss_pct:.0f}%" if loss_pct > 5 else "",
                        textposition="inside",
                        textfont=dict(
                            color="white", size=10, family="Inter, Arial, sans-serif"
                        ),
                        customdata=[data["loss"]],
                        hovertemplate="<b>Loss</b><br>"
                        + "Count: %{customdata}<br>"
                        + "Percentage: %{x:.1f}%<br>"
                        + "<extra></extra>",
                    ),
                    row=row_idx,
                    col=col_idx,
                )

                fig.update_yaxes(
                    row=row_idx,
                    col=col_idx,
                    showticklabels=False,
                    title_text="",
                )

            axis_number = (row_idx - 1) * 4 + 1
            yaxis_name = f"yaxis{'' if axis_number == 1 else axis_number}"
            y0, y1 = fig.layout[yaxis_name].domain
            y_mid = (y0 + y1) / 2

            fig.add_annotation(
                x=-0.03,
                y=y_mid,
                xref="paper",
                yref="paper",
                text=pretty_label(baseline_model),
                showarrow=False,
                xanchor="right",
                yanchor="middle",
                font=dict(size=12, color="#2C3E50"),
            )

            fig.add_annotation(
                x=1.03,
                y=y_mid,
                xref="paper",
                yref="paper",
                text=pretty_label(competitor),
                showarrow=False,
                xanchor="left",
                yanchor="middle",
                font=dict(size=12, color="#2C3E50"),
            )

        fig.update_layout(
            barmode="stack",
            title={
                "text": (
                    f"<b style='color:#2C3E50; font-size:18px;'>{pretty_label(baseline_model)} vs. others</b>"
                ),
                "x": 0.5,
                "xanchor": "center",
                "font": {"family": "Inter, Arial, sans-serif"},
            },
            height=max(200, 60 * len(competitor_models) + 80),
            width=1000,
            showlegend=False,
            plot_bgcolor="rgba(250,251,252,0.8)",
            paper_bgcolor="white",
            font=dict(
                family="Inter, -apple-system, BlinkMacSystemFont, 'Segoe UI', Arial, sans-serif",
                size=11,
                color="#2C3E50",
            ),
            margin=dict(l=150, r=150, t=100, b=20),
        )

        fig.update_xaxes(
            tickfont=dict(size=11, color="#34495E"),
            gridcolor="rgba(189,195,199,0.3)",
            gridwidth=1,
            zeroline=True,
            zerolinecolor="rgba(149,165,166,0.5)",
            zerolinewidth=1,
            range=[0, 100],
            ticksuffix="%",
            showticklabels=False,
        )

        fig.update_yaxes(
            tickfont=dict(size=11, color="#34495E"),
            gridcolor="rgba(189,195,199,0.2)",
            showticklabels=False,
        )

        for annotation in fig["layout"]["annotations"]:
            if "text" in annotation and annotation["text"] in macro_titles:
                annotation.update(
                    font=dict(
                        size=14, color="#2C3E50", family="Inter, Arial, sans-serif"
                    ),
                    bgcolor="rgba(255,255,255,0.9)",
                    bordercolor="rgba(189,195,199,0.3)",
                    borderwidth=1,
                    borderpad=4,
                )

        print(f"\nComparing {pretty_label(baseline_model)} (baseline) against:")
        for competitor in competitor_models:
            total_comparisons = sum(
                sum(win_loss_data[competitor][m].values()) for m in macro_cols
            )
            if total_comparisons > 0:
                baseline_wins = sum(
                    win_loss_data[competitor][m]["win"] for m in macro_cols
                )
                baseline_losses = sum(
                    win_loss_data[competitor][m]["loss"] for m in macro_cols
                )
                print(
                    f"  - vs {pretty_label(competitor)}: {baseline_wins} wins, {baseline_losses} losses across {total_comparisons // 4} images"
                )

        fig.show()

## Run Benchmark


In [126]:
BENCHMARK_CONFIG = {
    "models": [
        "gpt-4o-mini",
        "gpt-4o",
        "gemini/gemini-2.5-flash-preview-05-20",
        "gemini/gemini-2.5-pro-preview-06-05",
    ],
    "max_items": 10,
    "cache_dir": Path(".cache/food_scan_bench"),
    "max_concurrent_requests": 10,
    "prompt_variants": ["detailed", "step_by_step"],
    "use_embeddings_for_matching": True,
    "report_filename": "benchmark_results.csv",
}

results_df = await run_evaluation(
    models=BENCHMARK_CONFIG["models"],
    cache_dir=BENCHMARK_CONFIG["cache_dir"],
    max_items=BENCHMARK_CONFIG["max_items"],
    max_concurrent=BENCHMARK_CONFIG["max_concurrent_requests"],
    prompt_variants=BENCHMARK_CONFIG["prompt_variants"],
    use_embeddings=BENCHMARK_CONFIG["use_embeddings_for_matching"],
)


=== Running with prompt variant: 'detailed' ===
Using OpenAI embeddings semantic similarity


gpt-4o-mini (detailed)                  │          │ 0/10

gpt-4o (detailed)                       │          │ 0/10

gemini/gemini-2.5-flash-preview-05-20 (detailed)│          │ 0/10

gemini/gemini-2.5-pro-preview-06-05 (detailed)│          │ 0/10


=== Running with prompt variant: 'step_by_step' ===
Using OpenAI embeddings semantic similarity


gpt-4o-mini (step_by_step)              │          │ 0/10

gpt-4o (step_by_step)                   │          │ 0/10

gemini/gemini-2.5-flash-preview-05-20 (step_by_step)│          │ 0/10

gemini/gemini-2.5-pro-preview-06-05 (step_by_step)│          │ 0/10

In [143]:
analyzer = BenchmarkAnalyzer(results_df)

# analyzer.summary_statistics()
# analyzer.analyze_ingredient_matches()
# analyzer.statistical_significance_test()
# analyzer.create_performance_dashboard()
analyzer.macro_accuracy_heatmap()
analyzer.create_win_loss_analysis(baseline_model_name="gpt-4o")
# analyzer.export_detailed_report(BENCHMARK_CONFIG["report_filename"])

display(results_df.head())


=== WIN-TIE-LOSS ANALYSIS (Average Across Prompt Variants) ===

Comparing gpt-4o (baseline) against:
  - vs gemini-2.5-flash: 30 wins, 10 losses across 10 images
  - vs gemini-2.5-pro: 28 wins, 11 losses across 10 images
  - vs gpt-4o-mini: 19 wins, 18 losses across 10 images


,image_id,model,response_time_seconds,f1_ing,semantic_match,semantic_f1_ing,semantic_match_embeddings,ingredient_count_acc,mae_mac,error,cost_usd,calories_pct_error,carbs_pct_error,protein_pct_error,fat_pct_error,match_details
0,fsb_00000,gpt-4o-mini_detailed,14.22,0.500000,0.60,0.500000,0.60,0.600000,114.150,None,0.004074,95.838433,123.880597,9.634551,119.653179,"[{'gt_ingredient': 'scrambled eggs', 'pred_ing..."
1,fsb_00001,gpt-4o-mini_detailed,14.29,0.500000,0.50,0.750000,0.75,1.000000,12.725,None,0.005717,8.858859,11.340206,25.000000,40.776699,"[{'gt_ingredient': 'almond cake', 'pred_ingred..."
2,fsb_00002,gpt-4o-mini_detailed,13.08,0.000000,0.00,0.000000,0.00,0.333333,15.675,None,0.005673,9.354605,4.891304,138.095238,16.083916,[]
3,fsb_00003,gpt-4o-mini_detailed,12.75,0.333333,0.25,0.333333,0.25,0.500000,180.100,None,0.005671,68.881974,50.095969,80.769231,80.000000,"[{'gt_ingredient': 'pie crust', 'pred_ingredie..."
4,fsb_00004,gpt-4o-mini_detailed,13.70,0.000000,0.00,0.000000,0.00,0.666667,2.300,None,0.005684,6.567993,5.755396,27.272727,25.000000,[]
